In [5]:
# made by Gibeom LEE, HI LAB
# 1M to 100k.ipynb 와 달리, folder_path안에 있는 모든 csv파일을 전부 축소 시켜줘서, 다른 폴더에 저장해주는 코드입니다.

In [6]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

In [7]:
# 원본 CSV 파일이 있는 폴더 경로
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250917_40V 이상 다 찍기_기름별제외\물방울 부피별"

# find Peak 설정
height = 0.1  # 피크 감지 기준 전압 크기 (노이즈 레벨 고려)s
num_samples = 10000 # 최종 줄이고자 하는 행 수

In [8]:
# 결과를 저장할 새 폴더 경로 생성
parent_dir = os.path.dirname(folder_path)
original_folder_name = os.path.basename(folder_path)
new_folder_name = f"minimized_{original_folder_name}"
output_folder = os.path.join(parent_dir, new_folder_name)
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일 경로 수집
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file_path in csv_files:
    try:
        # CSV 파일 읽기 (헤더 없음, 앞 2줄 스킵)
        df = pd.read_csv(file_path, header=None, names=["Time", "Voltage"], skiprows=2)

        # 피크 검출
        peaks_positive, _ = find_peaks(df["Voltage"], height=height, distance=100)
        peaks_negative, _ = find_peaks(-df["Voltage"], height=height, distance=100)
        peaks = np.sort(np.concatenate((peaks_positive, peaks_negative)))

        # 피크 개수 확인
        num_peaks = len(peaks)
        print(f"{os.path.basename(file_path)} → Peak count: {num_peaks}")

        # 샘플링 인덱스 선택
        if num_peaks >= num_samples:
            selected_indices = np.sort(np.random.choice(peaks, num_samples, replace=False))
        else:
            num_non_peak_samples = num_samples - num_peaks
            non_peak_indices = np.setdiff1d(np.arange(len(df)), peaks)
            selected_non_peaks = np.linspace(0, len(non_peak_indices) - 1, num_non_peak_samples, dtype=int)
            selected_non_peaks = non_peak_indices[selected_non_peaks]

            selected_indices = np.sort(np.concatenate((peaks, selected_non_peaks)))

        # 축소된 데이터 생성
        filtered_df = df.iloc[selected_indices].reset_index(drop=True)
        # ---------------------------
        # 시간축 재설정 (0초부터 시작, 균일 간격 / endpoint=False)
        # ---------------------------
        t_start = df.iloc[0, 0]
        t_end = df.iloc[-1, 0]
        t_diff = t_end - t_start

        # 반올림된 전체 구간
        t_diff_rounded = round(t_diff)

        # 데이터 개수 (축소 후)
        N = len(filtered_df)

        # 균등 간격 생성 (마지막 점 제외)
        new_time = np.linspace(0, t_diff_rounded, N, endpoint=False)

        # 시간열 교체
        filtered_df["Time"] = new_time

        # ---------------------------
        # 저장
        # ---------------------------
        filename = os.path.basename(file_path)
        save_path = os.path.join(output_folder, filename)
        filtered_df.to_csv(save_path, index=False, header=False)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")


15ul_4cm.csv → Peak count: 5170
19ul_4cm.csv → Peak count: 5511
29ul_4cm.csv → Peak count: 5097
55ul_4cm.csv → Peak count: 7736
